## <span style="color : green"> Feature Engineering </span>

# <center> Table of Contents </center>

1. Split data into training and testing 
2. Drop independent variable based on - mean, median, variance
1. Remove outliers based on z-score and box-plot
1. correlationbetween independent variables
1. Correlation between independent variable and dependent
1. Select Features based on Chi-Square Test, SelectPercentile, VarianceThreshold
1. Feature Extraction using PCA and LDA
1. Mean of Pixel value of Image 